In [1]:
import redis
from dotenv import load_dotenv
import os
load_dotenv()



True

In [23]:
### This is direct connection pytho core redis
# Get Redis connection details from environment variables
# redis_user_id = os.getenv("REDIS_USER_ID")
redis_host = os.getenv("REDIS_HOST")
redis_port = os.getenv("REDIS_PORT")
redis_password = os.getenv("REDIS_PASSWORD")

# Create a Redis connection
redis_url = f"redis://default:{redis_password}@{redis_host}:{redis_port}"
redis_client = redis.StrictRedis.from_url(redis_url)

In [11]:
    ping_result = redis_client.ping()
    print(f"Connection to Redis successful: {ping_result}")

Connection to Redis successful: True


In [2]:
from langchain.vectorstores.redis import Redis

In [19]:
####For langchain methods
# redis_user_id = os.getenv("REDIS_USER_ID")
redis_host = os.getenv("REDIS_HOST")
redis_port = os.getenv("REDIS_PORT")
redis_password = os.getenv("REDIS_PASSWORD")
r=redis.Redis(host = redis_host,
              port=redis_port,
              password= redis_password)

In [6]:
r.ping()

True

In [12]:
print(r.keys())
print(redis_client.keys())

[]
[]


In [44]:
def load_pdf_to_redis(redis_host, redis_port, redis_password, pdf_path, pdf_key):
    try:
        # Create a Redis connection
        redis_url = f"redis://default:{redis_password}@{redis_host}:{redis_port}"
        redis_client = redis.StrictRedis.from_url(redis_url)
        
        # Get the full path to the PDF file in the /static folder
        static_folder = os.path.join(os.getcwd(), 'static')
        pdf_path = os.path.join(static_folder, pdf_filename)
        
        # Read the PDF file
        with open(pdf_path, 'rb') as pdf_file:
            pdf_content = pdf_file.read()

        # Store the PDF content in Redis
        redis_client.set(pdf_key, pdf_content)

        print(f"PDF '{pdf_key}' loaded into Redis successfully.")
    except Exception as e:
        print(f"Error loading PDF into Redis: {e}")

pdf_filename = "codigo_sustantivo_del_trabajo_colombia.pdf"  # Adjust the filename accordingly
pdf_key = "codigo_sustantivo_trabajo"

PDF 'codigo_sustantivo_trabajo' loaded into Redis successfully.


In [36]:
pdf_content_from_redis = redis_client.get(pdf_key)
if pdf_content_from_redis:
            # Puedes utilizar el contenido del PDF como desees
            # En este ejemplo, simplemente lo imprimimos
    print(pdf_content_from_redis.decode('utf-8'))
    else:
            print(f"Error: No se encontró el PDF con la clave '{pdf_key}' en Redis.")

Error al obtener el PDF desde Redis: Port could not be cast to integer value as 'redis_port'


In [45]:
import os
import redis
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings

# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
redis_host = os.getenv("REDIS_HOST")
redis_port = os.getenv("REDIS_PORT")
redis_password = os.getenv("REDIS_PASSWORD")

# Initialize Redis connection
redis_url = f"redis://default:{redis_password}@{redis_host}:{redis_port}"
redis_client = redis.StrictRedis.from_url(redis_url)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
embeddings = OpenAIEmbeddings()

def process_file(file_path):
    loader = None

    if file_path.endswith(".txt"):
        loader = TextLoader
    elif file_path.endswith(".pdf"):
        loader = PyPDFLoader

    if loader:
        documents = loader(file_path).load()
        docs = text_splitter.split_documents(documents)
        return docs
    else:
        return None

def save_embeddings_to_redis(docs, redis_key):
    try:
        for i, doc in enumerate(docs):
            # Extract text content from the document
            text_content = doc.page_content
            # Get embeddings for the text
            vector_representation = embeddings.get_vector(text_content)
            # Store the vector representation in Redis
            redis_client.set(f"{redis_key}_{i}", vector_representation.tobytes())
        print(f"Vector embeddings for key '{redis_key}' loaded into Redis successfully.")
    except Exception as e:
        print(f"Error loading embeddings into Redis: {e}")

# Example usage
file_path = os.path.join(static_folder, pdf_filename)
redis_key = "codigo_sustantivo_trabajo"

# Process the file and get the documents
docs = process_file(file_path)

if docs:
    # Save embeddings to Redis
    save_embeddings_to_redis(docs, redis_key)
else:
    print("Unsupported file format.")


NameError: name 'static_folder' is not defined